In [ ]:
def getExtension(data):
    URL=data[1].split('.')
    return URL[1]

def Get_handler(data):      #GET /index.html HTTP/1.0
    httpVersion = data[2]
    URL=data[1]
    URL = URL.split('/')[1]
    print(URL)      

    try: 
        #response = httpVersion + ' 200 OK\n\n'
        ext = getExtension(data)
        #connectionSocket.send(response.encode())
        #open file
        if ext=='png':
            f = open(URL,'rb')
        else :
            f = open(URL,'r')
        statinfo = os.stat(URL)
        size=statinfo.st_size
        now = datetime.now()
        response = httpVersion + ' 200 OK'
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        
        response=response+"\nDate: "+dt_string+"\nContent-Type: "+ext+"\nContent-Length: "+str(size)+"\n    \n"
        connectionSocket.send(response.encode())    
        #read first time
        l = f.read(1024)
        #loop and send
        while (l):
            if ext!='png':
                l=l.encode()
            connectionSocket.send(l)
            l = f.read(1024)
        f.close()
        print('Done sending')
    except FileNotFoundError:
        print(httpVersion + " 404 Not found")
        response = httpVersion + " 404 Not found"
        connectionSocket.send(response.encode()) 
    return 1

def Post_handler(data):
    URL=data[1].split('/')
    URL = URL[1]
    print(URL)
    ext = getExtension(data)

    if ext=='png' :
        f=open(URL,'wb') 
    else :
        f=open(URL,'w') 

    print('file opened')
    while True:
        print('receiving data')
        received_data = connectionSocket.recv(1024)
        if not received_data:
            break
        if ext!='png':
            received_data = received_data.decode()
        f.write(received_data)
        received_data=0
    f.close()
    print('Done receiving')
    
    return 1

# thread fuction 
def threaded(connectionSocket): 
    while True: 
        sentence = connectionSocket.recv(1024).decode()
        print(sentence)
        data = sentence.split()
        data=np.array(data)
        if data[0]=='GET':
            print("GET")
            Get_handler(data)
        else :
             print("POST")
             Post_handler(data)
     
        
              
            # lock released on exit 
        print_lock.release() 
        break
  
       
    connectionSocket.close() 
  

from socket import *
import numpy as np
from _thread import *
import threading 
from datetime import datetime  
print_lock = threading.Lock() 
import os
import matplotlib.image as img
serverPort = 12000
serverIP = 'localhost'

serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR, 1)
serverSocket.bind((serverIP,serverPort))
serverSocket.listen(1)
print("The server is ready to receive")

while True:
    connectionSocket, addr = serverSocket.accept()
    print_lock.acquire() 
    print('Connected to :', addr[0], ':', addr[1])
    start_new_thread(threaded, (connectionSocket,)) 
   
   
    

serverSocket.close()

The server is ready to receive
Connected to : 127.0.0.1 : 49504
POST /intro.txt HTTP/1.0
POST
intro.txt
file opened
receiving data
Done receiving
Connected to : 127.0.0.1 : 49510
POST /intro.txt HTTP/1.0
POST
intro.txt
file opened
receiving data
receiving data
Done receiving
Connected to : 127.0.0.1 : 49514
POST /file.txt HTTP/1.0 
POST
file.txt
file opened
receiving data
receiving data
Done receiving
Connected to : 127.0.0.1 : 49518
GET /int.txt HTTP/1.0
GET
int.txt
HTTP/1.0 404 Not found
